# Лабораторная работа №3
## Генерация текста с использованием LSTM (TensorFlow / Keras)

Обучающий текст загружается из внешнего файла `.txt`,
что позволяет легко расширять корпус данных.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam


## Обучающий текстовый файл

Создайте файл `train_text.txt` в той же директории,
где находится ноутбук, и поместите туда любой текст.


In [ ]:
with open("train_text.txt", "r", encoding="utf-8") as f:
    text = f.read()

text = text.lower()
print("Длина текста:", len(text))
print(text[:500])


Длина текста: 36507
5 этапов переживания расставания
конец отношений – это кризисный этап, в котором прошлые представления о себе или о вас, как о паре, больше не жизнеспособны, а новые только формируются.
практически все люди переживали расставание с любимым человеком и знают, что обычно это довольно болезненный процесс.

я знаю то чувство, которое охватывает после расставания, потому что и сама пережила разрывы с партнерами и часто сопровождаю клиентов на этом пути. сегодня поделюсь своими мыслями о том, почему б


## Построение словаря символов


In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

encoded = np.array([char_to_idx[c] for c in text])

print("Размер словаря:", vocab_size)


Размер словаря: 61


## Формирование обучающих последовательностей


In [ ]:
seq_length = 50
X = []
y = []

for i in range(len(encoded) - seq_length):
    X.append(encoded[i:i + seq_length])
    y.append(encoded[i + seq_length])

X = np.array(X)
y = np.array(y)

print("Количество обучающих примеров:", X.shape[0])


Количество обучающих примеров: 36457


## Архитектура LSTM-сети


In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64),
    LSTM(256),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy"
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Обучение модели


In [ ]:
model.fit(
    X,
    y,
    epochs=30,
    batch_size=64
)


Epoch 1/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 128s 220ms/step - loss: 3.0769
Epoch 2/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 130s 228ms/step - loss: 2.5433
Epoch 3/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 129s 226ms/step - loss: 2.4099
Epoch 4/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 125s 220ms/step - loss: 2.2775
Epoch 5/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 127s 223ms/step - loss: 2.1887
Epoch 6/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 131s 230ms/step - loss: 2.0867
Epoch 7/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 128s 224ms/step - loss: 1.9849
Epoch 8/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 139s 219ms/step - loss: 1.8922
Epoch 9/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 126s 221ms/step - loss: 1.8038
Epoch 10/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 143s 223ms/step - loss: 1.7186
Epoch 11/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 144s 226ms/step - loss: 1.6307
Epoch 12/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 127s 223ms/step - loss: 1.5394
Epoch 13/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 138s 216ms/step - loss: 1.4487
Epoch 14/30
570/570 ━━━━━━━━━━━━━━━━━━━━ 124s 218ms/step - loss: 1.3448
E

In [ ]:
## Генерация текста


In [ ]:
def generate_text(model, start_string, length=400):
    input_seq = [char_to_idx[c] for c in start_string.lower()]
    input_seq = tf.expand_dims(input_seq, 0)

    generated = start_string

    for _ in range(length):
        predictions = model(input_seq)
        predicted_id = tf.random.categorical(
            tf.math.log(predictions), 1
        )[0, 0].numpy()

        generated += idx_to_char[predicted_id]

        input_seq = tf.expand_dims(
            input_seq[0, 1:].numpy().tolist() + [predicted_id], 0
        )

    return generated


## Результат генерации текста


In [ ]:
generated_text = generate_text(
    model,
    start_string="Вино из одуванчиков ",
    length=500
)

print(generated_text)


Вино из одуванчиков — и в протают листье. дуглас вскридить спатишку карфый этих станок. на ночно стачит в себя роди. он завмернуться, просыпай зедлой день, полой в груди теперь, — прожном пыло себе на деревьев возом, ставание их вса, что обретилось обиты.
а не самовший первое зашивал сющев лет скаскованностушку друг, но бутяте. тан, завтория спогоди этап, к ного в спиной! рассыеные отвечхих, чтобы снежинаюсь, рэзвовну постига сосном, а показать мно ужен безужно завижаться пальцы в грати.

в его только прогуливаются


## Выводы

1. Обучающий текст вынесен во внешний файл `.txt`
2. Модель LSTM обучается на произвольном текстовом корпусе
3. Генерация демонстрирует сохранение статистических свойств текста
4. Увеличение объёма обучающего текста улучшает связность генерации
